In [1]:
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X

In [2]:
## Breast Cancer LASSO Exploration
## Prepare workspace
from scipy.io import loadmat
import numpy as np
X = loadmat("BreastCancer.mat")['X']
y = loadmat("BreastCancer.mat")['y']

##  10-fold CV 
# Set random seed for reproducibility
np.random.seed(42)

np.random.permutation(np.arange(X.shape[0]))

# Shuffle the data
permuted_indices = np.random.permutation(np.arange(X.shape[0]))
X = X[permuted_indices]
y = y[permuted_indices]

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,30],[31,60],[61,90],[91,120],[121,150],[151,179],[180,208],[209,237],[238,266],[267,295]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

# be sure to initiate the quantities you want to measure before looping
# through the various training, validation, and test partitions
lasso_error = []
squared_lasso_error = []
ridge_error = []
squared_ridge_error = []

# Loop over various cases
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(295))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = X[v1_ind,:]
    bv1 = y[v1_ind]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = X[v2_ind,:]
    bv2 = y[v2_ind]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = X[trn_ind,:]
    bt = y[trn_ind]
    
    print(len(v1_ind), len(v2_ind), len(trn_ind))
# Use training data to learn classifier
    W = ista_solve_hot(At,bt,np.logspace(-6,2))
    #print(W.shape)
    pred = np.sign(Av1 @ W)
    error = np.sum(1 - np.equal(pred, bv1), axis=0)
    best_lam = np.argmin(error)
    #print(best_lam)
    
    pred_2 = Av2 @ W[:,[best_lam]]
    error_2 = np.sum(1 - np.equal(np.sign(pred_2), bv2), axis= 0) / len(bv2)
    squared_error = np.linalg.norm(pred_2 - bv2, ord=2)
    
    lasso_error.append(error_2)
    squared_lasso_error.append(squared_error)
    
    
    for lam in np.logspace(-6,2):
        I = np.eye(At.shape[0])
        w_ridge = At.T@np.linalg.inv(At@At.T + lam * I)@ bt
        #print(w_ridge)
    pred = np.sign(Av1 @ w_ridge)
    error = np.sum(pred != np.sign(bv1), axis=0)
    best_ridge_lambda = np.argmin(error)
    
    ridge_pred = Av2 @ W[:, [best_ridge_lambda]]
    error_rate = np.sum(1 - np.equal(np.sign(ridge_pred), bv2), axis =0) / len(bv2)
    squared_error_rate = np.linalg.norm(ridge_pred - bv2, ord=2)
    
    ridge_error.append(error_rate)
    squared_ridge_error.append(squared_error_rate)
    
    
    
    
    
# Find best lambda value using first validation set, then evaluate
# performance on second validation set, and accumulate performance metrics
# over all cases partitions


30 30 235
30 30 235
30 30 235
30 30 235
30 29 236
29 29 237
29 29 237
29 30 236


In [5]:
print('lasso Error Rate')
print(np.sum(lasso_error) / np.size(lasso_error))
print('lasso Error Rate')
print(np.sum(squared_lasso_error) / np.size(squared_lasso_error))
print('Ridge Regression Error Rate')
print(np.sum(ridge_error ) / np.size(ridge_error ))
print('Ridge Regression Squared Error')
print(np.sum(squared_ridge_error) / np.size(squared_ridge_error))

0.2959770114942529
4.94365647171046
0.2959770114942529
4.974376682191677
